# Bank Marketing Campaign Success Prediction

## Executive Summary

This notebook presents a comprehensive analysis of bank marketing campaign data to predict customer subscription to term deposits. The dataset contains customer demographic information, financial details, and previous campaign interaction history. Our objective is to build a predictive model that can identify customers most likely to subscribe to a term deposit product.

## Problem Statement

Banks invest significant resources in marketing campaigns to promote their financial products. The ability to predict which customers are most likely to respond positively to marketing efforts can dramatically improve campaign efficiency and return on investment. This binary classification problem aims to predict whether a customer will subscribe to a term deposit based on their profile and interaction history.

## Dataset Overview

The dataset consists of the following components:
- Training data: 750,000 customer records with known outcomes
- Test data: 250,000 customer records for prediction
- Target variable: Binary indicator (0/1) for term deposit subscription
- Features: 16 attributes covering demographics, financial status, and campaign history

## Library Imports and Configuration

We begin by importing essential libraries for data manipulation, visualization, and statistical analysis. Each library serves a specific purpose in our exploratory data analysis workflow.

In [ ]:
# Core data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistical analysis
from scipy import stats
from scipy.stats import chi2_contingency, pearsonr

# Machine learning utilities
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Configuration for better visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## Data Loading and Initial Inspection

The first step in any data science project is to load and examine the structure of our dataset. We will load both training and test datasets and perform initial inspection to understand the data types, missing values, and basic statistics.

In [ ]:
# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# Display basic information about the datasets
print("Training Dataset Shape:", train_df.shape)
print("Test Dataset Shape:", test_df.shape)
print("Sample Submission Shape:", sample_submission.shape)
print("\n" + "="*50 + "\n")

# Display first few rows of training data
print("First 5 rows of training dataset:")
print(train_df.head())
print("\n" + "="*50 + "\n")

# Display first few rows of test data
print("First 5 rows of test dataset:")
print(test_df.head())

## Data Structure and Quality Assessment

Understanding the structure and quality of our data is crucial for effective analysis. We will examine data types, identify missing values, and assess the overall quality of each feature in our dataset.

In [ ]:
# Detailed information about the dataset structure
print("TRAINING DATASET INFORMATION")
print("="*50)
print(train_df.info())
print("\n" + "="*50 + "\n")

# Check for missing values
print("MISSING VALUES ANALYSIS")
print("="*50)
missing_train = train_df.isnull().sum()
missing_test = test_df.isnull().sum()

# Create separate analysis for train and test datasets
print("Training Dataset Missing Values:")
print("-" * 30)
for feature in train_df.columns:
    missing_count = missing_train[feature]
    missing_pct = (missing_count / len(train_df)) * 100
    print(f"{feature:15}: {missing_count:6,} ({missing_pct:5.2f}%)")

print("\nTest Dataset Missing Values:")
print("-" * 30)
for feature in test_df.columns:
    missing_count = missing_test[feature]
    missing_pct = (missing_count / len(test_df)) * 100
    print(f"{feature:15}: {missing_count:6,} ({missing_pct:5.2f}%)")

# Summary comparison (only for features present in both datasets)
common_features = [col for col in train_df.columns if col in test_df.columns]
missing_summary = pd.DataFrame({
    'Feature': common_features,
    'Train_Missing': [missing_train[col] for col in common_features],
    'Train_Missing_%': [(missing_train[col] / len(train_df) * 100).round(2) for col in common_features],
    'Test_Missing': [missing_test[col] for col in common_features],
    'Test_Missing_%': [(missing_test[col] / len(test_df) * 100).round(2) for col in common_features]
})

print("\nMissing Values Summary (Common Features):")
print("-" * 50)
print(missing_summary)
print("\n" + "="*50 + "\n")

# Data types summary
print("DATA TYPES SUMMARY")
print("="*50)
print(train_df.dtypes.value_counts())
print("\n" + "="*50 + "\n")

# Basic statistics for numerical features
print("NUMERICAL FEATURES SUMMARY")
print("="*50)
numerical_cols = train_df.select_dtypes(include=[np.number]).columns
print(train_df[numerical_cols].describe())

## Feature Definitions and Domain Understanding

Understanding the meaning and business context of each feature is essential for effective analysis. Below are the definitions of each feature in our dataset based on the bank marketing domain:

In [ ]:
# Feature definitions and categories
feature_definitions = {
    'id': 'Unique identifier for each customer',
    'age': 'Age of the customer in years',
    'job': 'Type of job/occupation of the customer',
    'marital': 'Marital status of the customer',
    'education': 'Education level of the customer',
    'default': 'Has credit in default (yes/no)',
    'balance': 'Average yearly balance in euros',
    'housing': 'Has housing loan (yes/no)',
    'loan': 'Has personal loan (yes/no)',
    'contact': 'Contact communication type',
    'day': 'Last contact day of the month',
    'month': 'Last contact month of year',
    'duration': 'Last contact duration in seconds',
    'campaign': 'Number of contacts performed during this campaign',
    'pdays': 'Number of days since previous campaign contact (-1 means never contacted)',
    'previous': 'Number of contacts performed before this campaign',
    'poutcome': 'Outcome of the previous marketing campaign',
    'y': 'Target variable - has the client subscribed to a term deposit (yes=1/no=0)'
}

# Categorize features by type
feature_categories = {
    'Demographic': ['age', 'job', 'marital', 'education'],
    'Financial': ['default', 'balance', 'housing', 'loan'],
    'Campaign_Contact': ['contact', 'day', 'month', 'duration'],
    'Campaign_History': ['campaign', 'pdays', 'previous', 'poutcome'],
    'Target': ['y']
}

print("FEATURE DEFINITIONS")
print("="*80)
for feature, definition in feature_definitions.items():
    print(f"{feature:12} : {definition}")

print("\n" + "="*80 + "\n")

print("FEATURE CATEGORIES")
print("="*80)
for category, features in feature_categories.items():
    print(f"{category:15} : {', '.join(features)}")

print("\n" + "="*80 + "\n")

# Identify categorical and numerical features
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()
numerical_features = train_df.select_dtypes(include=[np.number]).columns.tolist()

# Remove ID and target from analysis features
if 'id' in numerical_features:
    numerical_features.remove('id')
if 'y' in numerical_features:
    target_col = 'y'
    numerical_features.remove('y')

print("FEATURE TYPE CLASSIFICATION")
print("="*80)
print(f"Categorical Features ({len(categorical_features)}): {categorical_features}")
print(f"Numerical Features ({len(numerical_features)}): {numerical_features}")
print(f"Target Feature: {target_col}")

## Target Variable Analysis

The target variable represents the success of the marketing campaign. Understanding its distribution is crucial for model selection and evaluation strategy. We will analyze the class balance and implications for our modeling approach.

In [ ]:
# Target variable distribution
target_counts = train_df['y'].value_counts()
target_proportions = train_df['y'].value_counts(normalize=True)

print("TARGET VARIABLE DISTRIBUTION")
print("="*50)
print(f"Class 0 (No subscription): {target_counts[0]:,} ({target_proportions[0]:.2%})")
print(f"Class 1 (Subscription): {target_counts[1]:,} ({target_proportions[1]:.2%})")
print(f"Total samples: {target_counts.sum():,}")
print(f"Class imbalance ratio: {target_counts[0]/target_counts[1]:.2f}:1")

# Create visualization for target distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Bar plot
target_counts.plot(kind='bar', ax=axes[0], color=['lightcoral', 'lightblue'])
axes[0].set_title('Target Variable Distribution (Counts)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Target Class')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['No Subscription (0)', 'Subscription (1)'], rotation=0)

# Add count labels on bars
for i, v in enumerate(target_counts.values):
    axes[0].text(i, v + 5000, f'{v:,}', ha='center', va='bottom', fontweight='bold')

# Pie chart
axes[1].pie(target_counts.values, labels=['No Subscription (0)', 'Subscription (1)'], 
           autopct='%1.1f%%', startangle=90, colors=['lightcoral', 'lightblue'])
axes[1].set_title('Target Variable Distribution (Proportion)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Statistical significance check
print("\n" + "="*50)
print("CLASS IMBALANCE IMPLICATIONS:")
print("="*50)
if target_proportions[1] < 0.1:
    print("- Severely imbalanced dataset (minority class < 10%)")
    print("- Consider using stratified sampling and appropriate evaluation metrics")
    print("- SMOTE or other oversampling techniques may be beneficial")
elif target_proportions[1] < 0.3:
    print("- Moderately imbalanced dataset (minority class < 30%)")
    print("- Use stratified cross-validation and balanced evaluation metrics")
    print("- Consider class weights in model training")
else:
    print("- Relatively balanced dataset")
    print("- Standard evaluation approaches should work well")

## Categorical Features Analysis

Categorical features often contain valuable insights about customer segments and their relationship with the target variable. We will analyze the distribution of each categorical feature and examine their relationship with campaign success.

In [ ]:
# Analyze categorical features
print("CATEGORICAL FEATURES OVERVIEW")
print("="*80)

categorical_stats = []
for col in categorical_features:
    unique_count = train_df[col].nunique()
    most_common = train_df[col].mode()[0]
    most_common_freq = train_df[col].value_counts().iloc[0]
    most_common_pct = (most_common_freq / len(train_df)) * 100
    
    categorical_stats.append({
        'Feature': col,
        'Unique_Values': unique_count,
        'Most_Common': most_common,
        'Most_Common_Count': most_common_freq,
        'Most_Common_Percentage': round(most_common_pct, 2)
    })

categorical_df = pd.DataFrame(categorical_stats)
print(categorical_df)

print("\n" + "="*80 + "\n")

# Detailed analysis of each categorical feature
for col in categorical_features:
    print(f"FEATURE: {col.upper()}")
    print("-" * 40)
    
    value_counts = train_df[col].value_counts()
    print("Value distribution:")
    for value, count in value_counts.items():
        percentage = (count / len(train_df)) * 100
        print(f"  {value}: {count:,} ({percentage:.1f}%)")
    print()

# Create visualizations for categorical features
fig, axes = plt.subplots(2, 4, figsize=(20, 12))
axes = axes.ravel()

for i, col in enumerate(categorical_features):
    if i < len(axes):
        value_counts = train_df[col].value_counts()
        
        # Create bar plot
        value_counts.plot(kind='bar', ax=axes[i], color='lightblue', alpha=0.7)
        axes[i].set_title(f'{col.title()} Distribution', fontweight='bold')
        axes[i].set_xlabel(col.title())
        axes[i].set_ylabel('Count')
        axes[i].tick_params(axis='x', rotation=45)
        
        # Add count labels on bars
        for j, v in enumerate(value_counts.values):
            if v > 1000:  # Only show labels for significant bars
                axes[i].text(j, v + 1000, f'{v:,}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

## Numerical Features Analysis

Numerical features provide quantitative insights into customer characteristics and campaign interactions. We will examine their distributions, identify outliers, and assess their statistical properties.

In [ ]:
# Detailed analysis of numerical features
print("NUMERICAL FEATURES STATISTICAL SUMMARY")
print("="*80)
print(train_df[numerical_features].describe())

print("\n" + "="*80 + "\n")

# Check for outliers using IQR method
print("OUTLIER ANALYSIS")
print("="*80)

outlier_summary = []
for col in numerical_features:
    Q1 = train_df[col].quantile(0.25)
    Q3 = train_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = train_df[(train_df[col] < lower_bound) | (train_df[col] > upper_bound)][col]
    outlier_count = len(outliers)
    outlier_percentage = (outlier_count / len(train_df)) * 100
    
    outlier_summary.append({
        'Feature': col,
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'Lower_Bound': lower_bound,
        'Upper_Bound': upper_bound,
        'Outlier_Count': outlier_count,
        'Outlier_Percentage': round(outlier_percentage, 2)
    })

outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df)

# Distribution analysis with histograms
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, col in enumerate(numerical_features):
    if i < len(axes):
        # Create histogram
        axes[i].hist(train_df[col], bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
        axes[i].set_title(f'{col.title()} Distribution', fontweight='bold')
        axes[i].set_xlabel(col.title())
        axes[i].set_ylabel('Frequency')
        axes[i].grid(True, alpha=0.3)
        
        # Add statistical information
        mean_val = train_df[col].mean()
        median_val = train_df[col].median()
        axes[i].axvline(mean_val, color='red', linestyle='--', label=f'Mean: {mean_val:.2f}')
        axes[i].axvline(median_val, color='blue', linestyle='--', label=f'Median: {median_val:.2f}')
        axes[i].legend()

# Remove empty subplots
for i in range(len(numerical_features), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

# Box plots for outlier visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, col in enumerate(numerical_features):
    if i < len(axes):
        axes[i].boxplot(train_df[col])
        axes[i].set_title(f'{col.title()} Box Plot', fontweight='bold')
        axes[i].set_ylabel(col.title())
        axes[i].grid(True, alpha=0.3)

# Remove empty subplots
for i in range(len(numerical_features), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

## Bivariate Analysis: Features vs Target

Understanding the relationship between individual features and the target variable is crucial for feature selection and model interpretation. We will examine how each feature correlates with campaign success.

In [ ]:
# Categorical features vs Target analysis
print("CATEGORICAL FEATURES vs TARGET ANALYSIS")
print("="*80)

categorical_target_analysis = []

for col in categorical_features:
    # Create crosstab
    crosstab = pd.crosstab(train_df[col], train_df['y'], normalize='index')
    
    # Chi-square test for independence
    chi2, p_value, dof, expected = chi2_contingency(pd.crosstab(train_df[col], train_df['y']))
    
    # Calculate success rate for each category
    success_rates = train_df.groupby(col)['y'].agg(['count', 'sum', 'mean']).round(4)
    success_rates['success_rate_pct'] = (success_rates['mean'] * 100).round(2)
    
    print(f"\nFEATURE: {col.upper()}")
    print("-" * 50)
    print("Success rates by category:")
    print(success_rates)
    print(f"Chi-square test p-value: {p_value:.6f}")
    
    if p_value < 0.05:
        print("Result: Statistically significant association with target")
    else:
        print("Result: No significant association with target")
    
    # Store analysis results
    categorical_target_analysis.append({
        'Feature': col,
        'Chi2_statistic': chi2,
        'P_value': p_value,
        'Significant': p_value < 0.05,
        'Max_success_rate': success_rates['mean'].max(),
        'Min_success_rate': success_rates['mean'].min(),
        'Rate_difference': success_rates['mean'].max() - success_rates['mean'].min()
    })

# Create summary dataframe
categorical_analysis_df = pd.DataFrame(categorical_target_analysis)
categorical_analysis_df = categorical_analysis_df.sort_values('Rate_difference', ascending=False)

print("\n" + "="*80)
print("CATEGORICAL FEATURES IMPORTANCE RANKING")
print("="*80)
print(categorical_analysis_df)

# Visualize categorical features vs target
fig, axes = plt.subplots(2, 4, figsize=(20, 12))
axes = axes.ravel()

for i, col in enumerate(categorical_features):
    if i < len(axes):
        # Calculate success rates
        success_rates = train_df.groupby(col)['y'].mean().sort_values(ascending=False)
        
        # Create bar plot
        success_rates.plot(kind='bar', ax=axes[i], color='lightcoral', alpha=0.7)
        axes[i].set_title(f'{col.title()} vs Success Rate', fontweight='bold')
        axes[i].set_xlabel(col.title())
        axes[i].set_ylabel('Success Rate')
        axes[i].tick_params(axis='x', rotation=45)
        axes[i].grid(True, alpha=0.3)
        
        # Add value labels on bars
        for j, v in enumerate(success_rates.values):
            axes[i].text(j, v + 0.005, f'{v:.2%}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
# Numerical features vs Target analysis
print("\n" + "="*80)
print("NUMERICAL FEATURES vs TARGET ANALYSIS")
print("="*80)

numerical_target_analysis = []

for col in numerical_features:
    # Calculate correlation with target
    correlation = train_df[col].corr(train_df['y'])
    
    # Statistical comparison between success and failure groups
    success_group = train_df[train_df['y'] == 1][col]
    failure_group = train_df[train_df['y'] == 0][col]
    
    # T-test for difference in means
    t_stat, p_value = stats.ttest_ind(success_group, failure_group)
    
    # Calculate statistics for each group
    success_stats = {
        'mean': success_group.mean(),
        'median': success_group.median(),
        'std': success_group.std()
    }
    
    failure_stats = {
        'mean': failure_group.mean(),
        'median': failure_group.median(),
        'std': failure_group.std()
    }
    
    print(f"\nFEATURE: {col.upper()}")
    print("-" * 50)
    print(f"Correlation with target: {correlation:.4f}")
    print(f"Success group - Mean: {success_stats['mean']:.2f}, Median: {success_stats['median']:.2f}")
    print(f"Failure group - Mean: {failure_stats['mean']:.2f}, Median: {failure_stats['median']:.2f}")
    print(f"T-test p-value: {p_value:.6f}")
    
    if p_value < 0.05:
        print("Result: Statistically significant difference between groups")
    else:
        print("Result: No significant difference between groups")
    
    numerical_target_analysis.append({
        'Feature': col,
        'Correlation': correlation,
        'T_test_pvalue': p_value,
        'Significant': p_value < 0.05,
        'Success_mean': success_stats['mean'],
        'Failure_mean': failure_stats['mean'],
        'Mean_difference': success_stats['mean'] - failure_stats['mean']
    })

# Create summary dataframe
numerical_analysis_df = pd.DataFrame(numerical_target_analysis)
numerical_analysis_df = numerical_analysis_df.sort_values('Correlation', key=abs, ascending=False)

print("\n" + "="*80)
print("NUMERICAL FEATURES IMPORTANCE RANKING")
print("="*80)
print(numerical_analysis_df)

# Visualize numerical features vs target with box plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, col in enumerate(numerical_features):
    if i < len(axes):
        # Create box plot grouped by target
        data_to_plot = [train_df[train_df['y'] == 0][col].values, 
                       train_df[train_df['y'] == 1][col].values]
        
        box_plot = axes[i].boxplot(data_to_plot, labels=['No Success (0)', 'Success (1)'])
        axes[i].set_title(f'{col.title()} by Target', fontweight='bold')
        axes[i].set_xlabel('Target')
        axes[i].set_ylabel(col.title())
        axes[i].grid(True, alpha=0.3)

# Remove empty subplots
for i in range(len(numerical_features), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

## Correlation Analysis and Feature Relationships

Understanding the relationships between features helps identify multicollinearity and potential feature interactions. This analysis guides our feature engineering and model selection decisions.

In [ ]:
# Correlation analysis for numerical features
numerical_features_with_target = numerical_features + ['y']
correlation_matrix = train_df[numerical_features_with_target].corr()

print("CORRELATION MATRIX")
print("="*80)
print(correlation_matrix.round(4))

# Find highly correlated feature pairs
print("\n" + "="*80)
print("HIGHLY CORRELATED FEATURE PAIRS (|correlation| > 0.5)")
print("="*80)

high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.5 and correlation_matrix.columns[i] != 'y' and correlation_matrix.columns[j] != 'y':
            high_corr_pairs.append({
                'Feature_1': correlation_matrix.columns[i],
                'Feature_2': correlation_matrix.columns[j],
                'Correlation': corr_value
            })

if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs)
    high_corr_df = high_corr_df.sort_values('Correlation', key=abs, ascending=False)
    print(high_corr_df)
else:
    print("No highly correlated feature pairs found.")

# Visualize correlation matrix
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='RdBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .5}, fmt='.3f')
plt.title('Correlation Matrix of Numerical Features', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Feature importance based on correlation with target
target_correlations = correlation_matrix['y'].drop('y').sort_values(key=abs, ascending=False)

print("\n" + "="*80)
print("FEATURE IMPORTANCE BASED ON TARGET CORRELATION")
print("="*80)
print("Features ranked by absolute correlation with target:")
for feature, corr in target_correlations.items():
    print(f"{feature:15}: {corr:7.4f}")

# Visualize feature correlations with target
plt.figure(figsize=(12, 8))
target_correlations.plot(kind='barh', color=['red' if x < 0 else 'blue' for x in target_correlations.values])
plt.title('Feature Correlations with Target Variable', fontsize=16, fontweight='bold')
plt.xlabel('Correlation with Target')
plt.ylabel('Features')
plt.grid(True, alpha=0.3)
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.tight_layout()
plt.show()

## Special Values and Data Quality Issues

Banking datasets often contain special values that require careful handling. We will examine specific patterns in our data that may indicate missing values, data collection artifacts, or domain-specific encoding.

In [ ]:
# Analyze special values and patterns
print("SPECIAL VALUES ANALYSIS")
print("="*80)

# Check for -1 values in pdays (indicates no previous contact)
pdays_negative = (train_df['pdays'] == -1).sum()
pdays_negative_pct = (pdays_negative / len(train_df)) * 100

print(f"Records with pdays = -1 (no previous contact): {pdays_negative:,} ({pdays_negative_pct:.1f}%)")

# Analyze 'unknown' values in categorical features
print("\n'UNKNOWN' VALUES IN CATEGORICAL FEATURES:")
print("-" * 50)
for col in categorical_features:
    unknown_count = (train_df[col] == 'unknown').sum()
    unknown_pct = (unknown_count / len(train_df)) * 100
    print(f"{col:15}: {unknown_count:,} ({unknown_pct:.1f}%)")

# Check for zero values in numerical features
print("\nZERO VALUES IN NUMERICAL FEATURES:")
print("-" * 50)
for col in numerical_features:
    zero_count = (train_df[col] == 0).sum()
    zero_pct = (zero_count / len(train_df)) * 100
    print(f"{col:15}: {zero_count:,} ({zero_pct:.1f}%)")

# Analyze negative balance values
negative_balance = (train_df['balance'] < 0).sum()
negative_balance_pct = (negative_balance / len(train_df)) * 100
print(f"\nRecords with negative balance: {negative_balance:,} ({negative_balance_pct:.1f}%)")

# Check for extreme values
print("\nEXTREME VALUES ANALYSIS:")
print("-" * 50)
for col in numerical_features:
    min_val = train_df[col].min()
    max_val = train_df[col].max()
    mean_val = train_df[col].mean()
    std_val = train_df[col].std()
    
    # Values beyond 3 standard deviations
    extreme_lower = (train_df[col] < (mean_val - 3 * std_val)).sum()
    extreme_upper = (train_df[col] > (mean_val + 3 * std_val)).sum()
    total_extreme = extreme_lower + extreme_upper
    extreme_pct = (total_extreme / len(train_df)) * 100
    
    print(f"{col:15}: Min={min_val:8.2f}, Max={max_val:8.2f}, Extreme values: {total_extreme:,} ({extreme_pct:.2f}%)")

# Analyze the relationship between special values and target
print("\n" + "="*80)
print("SPECIAL VALUES vs TARGET ANALYSIS")
print("="*80)

# pdays = -1 vs target
no_previous_contact = train_df[train_df['pdays'] == -1]
previous_contact = train_df[train_df['pdays'] != -1]

print(f"Success rate with no previous contact: {no_previous_contact['y'].mean():.4f}")
print(f"Success rate with previous contact: {previous_contact['y'].mean():.4f}")

# Unknown values vs target for each categorical feature
print("\nSUCCESS RATES FOR 'UNKNOWN' vs 'KNOWN' VALUES:")
print("-" * 60)
for col in categorical_features:
    unknown_success = train_df[train_df[col] == 'unknown']['y'].mean()
    known_success = train_df[train_df[col] != 'unknown']['y'].mean()
    print(f"{col:15}: Unknown={unknown_success:.4f}, Known={known_success:.4f}")

# Visualize special values impact
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# pdays distribution
axes[0, 0].hist(train_df[train_df['pdays'] != -1]['pdays'], bins=50, alpha=0.7, color='lightblue')
axes[0, 0].set_title('Distribution of pdays (excluding -1 values)', fontweight='bold')
axes[0, 0].set_xlabel('Days since previous contact')
axes[0, 0].set_ylabel('Frequency')

# Balance distribution including negatives
axes[0, 1].hist(train_df['balance'], bins=50, alpha=0.7, color='lightgreen')
axes[0, 1].set_title('Balance Distribution (including negatives)', fontweight='bold')
axes[0, 1].set_xlabel('Balance')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(x=0, color='red', linestyle='--', label='Zero balance')
axes[0, 1].legend()

# Unknown values count by feature
unknown_counts = []
for col in categorical_features:
    unknown_counts.append((train_df[col] == 'unknown').sum())

axes[1, 0].bar(categorical_features, unknown_counts, color='lightcoral', alpha=0.7)
axes[1, 0].set_title('Count of "Unknown" Values by Feature', fontweight='bold')
axes[1, 0].set_xlabel('Features')
axes[1, 0].set_ylabel('Count of Unknown Values')
axes[1, 0].tick_params(axis='x', rotation=45)

# Success rate comparison for pdays
pdays_groups = ['No Previous Contact', 'Previous Contact']
pdays_success_rates = [no_previous_contact['y'].mean(), previous_contact['y'].mean()]

axes[1, 1].bar(pdays_groups, pdays_success_rates, color=['lightblue', 'lightgreen'], alpha=0.7)
axes[1, 1].set_title('Success Rate: Previous Contact vs No Previous Contact', fontweight='bold')
axes[1, 1].set_xlabel('Contact History')
axes[1, 1].set_ylabel('Success Rate')

# Add value labels on bars
for i, v in enumerate(pdays_success_rates):
    axes[1, 1].text(i, v + 0.005, f'{v:.2%}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## Key Insights and Conclusions from EDA

Based on our comprehensive exploratory data analysis, we have identified several important patterns and insights that will guide our modeling approach.

In [ ]:
# Summarize key insights from EDA
print("KEY INSIGHTS FROM EXPLORATORY DATA ANALYSIS")
print("="*80)

print("\n1. DATASET CHARACTERISTICS:")
print("-" * 40)
print(f"   - Training samples: {len(train_df):,}")
print(f"   - Test samples: {len(test_df):,}")
print(f"   - Features: {len(train_df.columns)-1} (excluding target)")
print(f"   - Target distribution: {train_df['y'].value_counts()[0]:,} negative, {train_df['y'].value_counts()[1]:,} positive")
print(f"   - Class imbalance ratio: {train_df['y'].value_counts()[0]/train_df['y'].value_counts()[1]:.1f}:1")

print("\n2. DATA QUALITY:")
print("-" * 40)
print("   - No missing values detected in standard sense")
print("   - Special encoding patterns identified:")
print(f"     * 'unknown' values in categorical features")
print(f"     * -1 values in 'pdays' (no previous contact)")
print(f"     * Negative balance values present")

print("\n3. MOST PREDICTIVE FEATURES (based on target correlation/association):")
print("-" * 40)

# Get top categorical features by chi-square significance and rate difference
if 'categorical_analysis_df' in locals():
    top_categorical = categorical_analysis_df.head(3)['Feature'].tolist()
    print(f"   Categorical: {', '.join(top_categorical)}")

# Get top numerical features by correlation
if 'target_correlations' in locals():
    top_numerical = target_correlations.head(3).index.tolist()
    print(f"   Numerical: {', '.join(top_numerical)}")

print("\n4. POTENTIAL DATA ISSUES:")
print("-" * 40)
print("   - High percentage of 'unknown' values in some categorical features")
print("   - Outliers present in numerical features")
print("   - Potential multicollinearity between related features")

print("\n5. BUSINESS INSIGHTS:")
print("-" * 40)
print("   - Previous campaign outcome strongly influences success")
print("   - Contact method and timing appear important")
print("   - Customer demographics show varying success rates")
print("   - Financial status indicators may be predictive")

print("\n" + "="*80)
print("RECOMMENDED NEXT STEPS")
print("="*80)

next_steps = [
    "Feature Engineering:",
    "  - Handle 'unknown' values appropriately (separate category vs imputation)",
    "  - Create interaction features between highly predictive variables",
    "  - Engineer temporal features from month/day information",
    "  - Create binned versions of continuous variables",
    "  - Develop campaign intensity and customer lifecycle features",
    "",
    "Data Preprocessing:",
    "  - Implement appropriate scaling for numerical features",
    "  - Encode categorical variables (target encoding for high cardinality)",
    "  - Handle outliers through capping or transformation",
    "  - Address class imbalance through sampling or class weights",
    "",
    "Model Development:",
    "  - Start with gradient boosting models (LightGBM, XGBoost)",
    "  - Implement proper cross-validation strategy",
    "  - Build ensemble of diverse models",
    "  - Focus on features showing strong target association",
    "",
    "Validation Strategy:",
    "  - Use stratified cross-validation due to class imbalance",
    "  - Monitor for overfitting with early stopping",
    "  - Validate on multiple metrics (AUC, precision, recall)",
    "  - Perform adversarial validation for train/test similarity"
]

for step in next_steps:
    print(step)

# Feature Engineering and Data Preprocessing

Based on our comprehensive EDA, we now implement targeted feature engineering strategies to improve model performance. Our analysis revealed key patterns in special values, categorical relationships, and numerical distributions that guide our preprocessing approach.

## Feature Engineering Strategy

Our feature engineering approach focuses on:
1. Handling special values and unknown categories appropriately
2. Creating meaningful interaction features based on domain knowledge
3. Engineering temporal and cyclical features from date information
4. Developing customer segmentation and campaign intensity metrics
5. Addressing class imbalance and outlier management

## Data Preparation and Copy Creation

We start by creating working copies of our datasets to preserve the original data and prepare for feature engineering transformations.

In [ ]:
# Create working copies of the datasets
train_processed = train_df.copy()
test_processed = test_df.copy()

print("DATASET PREPARATION")
print("="*50)
print(f"Training dataset shape: {train_processed.shape}")
print(f"Test dataset shape: {test_processed.shape}")

# Store original feature lists for reference
original_features = [col for col in train_processed.columns if col not in ['id', 'y']]
print(f"Original features count: {len(original_features)}")
print(f"Original features: {original_features}")

# Separate features and target
X_train = train_processed.drop(['y'], axis=1)
y_train = train_processed['y']
X_test = test_processed.copy()

print(f"\nFeature matrix shape: {X_train.shape}")
print(f"Target vector shape: {y_train.shape}")
print(f"Test matrix shape: {X_test.shape}")

## Special Values and Missing Data Handling

Based on our EDA findings, we identified several special encoding patterns that require careful handling. We will create indicator features for these patterns and transform them appropriately.

In [ ]:
def handle_special_values(df_train, df_test):
    """
    Handle special values and create indicator features
    """
    # Create copies to avoid modifying original data
    train_clean = df_train.copy()
    test_clean = df_test.copy()
    
    print("HANDLING SPECIAL VALUES")
    print("="*50)
    
    # 1. Handle pdays = -1 (no previous contact)
    print("1. Processing 'pdays' feature:")
    print(f"   - Records with pdays = -1: {(train_clean['pdays'] == -1).sum():,}")
    
    # Create indicator for no previous contact
    train_clean['no_previous_contact'] = (train_clean['pdays'] == -1).astype(int)
    test_clean['no_previous_contact'] = (test_clean['pdays'] == -1).astype(int)
    
    # Transform pdays: replace -1 with a reasonable value (e.g., 999 or median of positive values)
    pdays_positive_median = train_clean[train_clean['pdays'] > 0]['pdays'].median()
    train_clean['pdays_transformed'] = train_clean['pdays'].replace(-1, 999)  # Large value indicating "no contact"
    test_clean['pdays_transformed'] = test_clean['pdays'].replace(-1, 999)
    
    print(f"   - Created 'no_previous_contact' indicator")
    print(f"   - Created 'pdays_transformed' with -1 replaced by 999")
    
    # 2. Handle 'unknown' values in categorical features
    print("\n2. Processing 'unknown' values in categorical features:")
    categorical_cols = train_clean.select_dtypes(include=['object']).columns
    
    for col in categorical_cols:
        unknown_count = (train_clean[col] == 'unknown').sum()
        unknown_pct = (unknown_count / len(train_clean)) * 100
        
        if unknown_count > 0:
            print(f"   - {col}: {unknown_count:,} unknown values ({unknown_pct:.1f}%)")
            
            # Create indicator for unknown values
            train_clean[f'{col}_is_unknown'] = (train_clean[col] == 'unknown').astype(int)
            test_clean[f'{col}_is_unknown'] = (test_clean[col] == 'unknown').astype(int)
            
            # Keep 'unknown' as a separate category (don't impute)
            # This preserves the information that the value was unknown
    
    # 3. Handle negative balance
    print("\n3. Processing negative balance values:")
    negative_balance_count = (train_clean['balance'] < 0).sum()
    print(f"   - Records with negative balance: {negative_balance_count:,}")
    
    # Create indicator for negative balance
    train_clean['has_negative_balance'] = (train_clean['balance'] < 0).astype(int)
    test_clean['has_negative_balance'] = (test_clean['balance'] < 0).astype(int)
    
    # Create absolute balance feature
    train_clean['balance_abs'] = train_clean['balance'].abs()
    test_clean['balance_abs'] = test_clean['balance'].abs()
    
    print(f"   - Created 'has_negative_balance' indicator")
    print(f"   - Created 'balance_abs' with absolute values")
    
    # 4. Handle zero values in key features
    print("\n4. Processing zero values:")
    zero_features = ['duration', 'campaign', 'previous']
    
    for col in zero_features:
        zero_count = (train_clean[col] == 0).sum()
        if zero_count > 0:
            zero_pct = (zero_count / len(train_clean)) * 100
            print(f"   - {col}: {zero_count:,} zero values ({zero_pct:.1f}%)")
            
            # Create indicator for zero values
            train_clean[f'{col}_is_zero'] = (train_clean[col] == 0).astype(int)
            test_clean[f'{col}_is_zero'] = (test_clean[col] == 0).astype(int)
    
    return train_clean, test_clean

# Apply special values handling
X_train_processed, X_test_processed = handle_special_values(X_train, X_test)

print(f"\nSHAPE AFTER SPECIAL VALUES HANDLING:")
print(f"Training set: {X_train_processed.shape}")
print(f"Test set: {X_test_processed.shape}")

# Show new features created
original_cols = set(X_train.columns)
new_cols = set(X_train_processed.columns) - original_cols
print(f"\nNew features created: {len(new_cols)}")
print(f"New features: {sorted(list(new_cols))}")

## Temporal and Cyclical Feature Engineering

Time-based features often contain valuable patterns in marketing campaigns. We will engineer features from the 'month' and 'day' information to capture seasonal and cyclical patterns.

In [ ]:
def create_temporal_features(df_train, df_test):
    """
    Create temporal and cyclical features from month and day information
    """
    train_temp = df_train.copy()
    test_temp = df_test.copy()
    
    print("CREATING TEMPORAL FEATURES")
    print("="*50)
    
    # Month mappings
    month_mapping = {
        'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
        'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
    }
    
    # 1. Convert month names to numbers
    train_temp['month_num'] = train_temp['month'].map(month_mapping)
    test_temp['month_num'] = test_temp['month'].map(month_mapping)
    
    # 2. Create cyclical features for month (captures seasonality)
    train_temp['month_sin'] = np.sin(2 * np.pi * train_temp['month_num'] / 12)
    train_temp['month_cos'] = np.cos(2 * np.pi * train_temp['month_num'] / 12)
    test_temp['month_sin'] = np.sin(2 * np.pi * test_temp['month_num'] / 12)
    test_temp['month_cos'] = np.cos(2 * np.pi * test_temp['month_num'] / 12)
    
    # 3. Create cyclical features for day (captures monthly cycles)
    train_temp['day_sin'] = np.sin(2 * np.pi * train_temp['day'] / 31)
    train_temp['day_cos'] = np.cos(2 * np.pi * train_temp['day'] / 31)
    test_temp['day_sin'] = np.sin(2 * np.pi * test_temp['day'] / 31)
    test_temp['day_cos'] = np.cos(2 * np.pi * test_temp['day'] / 31)
    
    # 4. Create seasonal categories
    def get_season(month_num):
        if month_num in [12, 1, 2]:
            return 'winter'
        elif month_num in [3, 4, 5]:
            return 'spring'
        elif month_num in [6, 7, 8]:
            return 'summer'
        else:
            return 'autumn'
    
    train_temp['season'] = train_temp['month_num'].apply(get_season)
    test_temp['season'] = test_temp['month_num'].apply(get_season)
    
    # 5. Create day categories
    def get_day_category(day):
        if day <= 10:
            return 'early_month'
        elif day <= 20:
            return 'mid_month'
        else:
            return 'late_month'
    
    train_temp['day_category'] = train_temp['day'].apply(get_day_category)
    test_temp['day_category'] = test_temp['day'].apply(get_day_category)
    
    # 6. Create quarter feature
    train_temp['quarter'] = ((train_temp['month_num'] - 1) // 3) + 1
    test_temp['quarter'] = ((test_temp['month_num'] - 1) // 3) + 1
    
    print("Created temporal features:")
    print("- month_num: Numerical month (1-12)")
    print("- month_sin, month_cos: Cyclical month encoding")
    print("- day_sin, day_cos: Cyclical day encoding")
    print("- season: Seasonal categories")
    print("- day_category: Early/mid/late month categories")
    print("- quarter: Quarter of the year (1-4)")
    
    return train_temp, test_temp

# Apply temporal feature engineering
X_train_processed, X_test_processed = create_temporal_features(X_train_processed, X_test_processed)

print(f"\nSHAPE AFTER TEMPORAL FEATURE ENGINEERING:")
print(f"Training set: {X_train_processed.shape}")
print(f"Test set: {X_test_processed.shape}")

# Show temporal feature distribution
print(f"\nTEMPORAL FEATURE DISTRIBUTIONS:")
print("Season distribution:")
print(X_train_processed['season'].value_counts())
print("\nDay category distribution:")
print(X_train_processed['day_category'].value_counts())
print("\nQuarter distribution:")
print(X_train_processed['quarter'].value_counts())

## Customer Segmentation and Domain-Specific Features

Based on banking domain knowledge, we will create features that capture customer profiles, financial status, and campaign interaction patterns that are relevant for term deposit marketing.

In [ ]:
def create_customer_features(df_train, df_test):
    """
    Create customer segmentation and domain-specific features
    """
    train_cust = df_train.copy()
    test_cust = df_test.copy()
    
    print("CREATING CUSTOMER SEGMENTATION FEATURES")
    print("="*50)
    
    # 1. Age-based segmentation
    def categorize_age(age):
        if age <= 25:
            return 'young'
        elif age <= 35:
            return 'young_adult'
        elif age <= 50:
            return 'middle_aged'
        elif age <= 65:
            return 'senior'
        else:
            return 'elderly'
    
    train_cust['age_group'] = train_cust['age'].apply(categorize_age)
    test_cust['age_group'] = test_cust['age'].apply(categorize_age)
    
    # 2. Balance-based segmentation
    def categorize_balance(balance):
        if balance < 0:
            return 'negative'
        elif balance == 0:
            return 'zero'
        elif balance <= 1000:
            return 'low'
        elif balance <= 5000:
            return 'medium'
        elif balance <= 20000:
            return 'high'
        else:
            return 'very_high'
    
    train_cust['balance_category'] = train_cust['balance'].apply(categorize_balance)
    test_cust['balance_category'] = test_cust['balance'].apply(categorize_balance)
    
    # 3. Financial stability score
    # Combine multiple financial indicators
    train_cust['financial_stability'] = (
        (train_cust['default'] == 'no').astype(int) * 2 +
        (train_cust['balance'] > 0).astype(int) +
        (train_cust['housing'] == 'yes').astype(int) +
        (train_cust['loan'] == 'no').astype(int)
    )
    test_cust['financial_stability'] = (
        (test_cust['default'] == 'no').astype(int) * 2 +
        (test_cust['balance'] > 0).astype(int) +
        (test_cust['housing'] == 'yes').astype(int) +
        (test_cust['loan'] == 'no').astype(int)
    )
    
    # 4. Campaign intensity metrics
    # Total campaign exposure
    train_cust['total_contacts'] = train_cust['campaign'] + train_cust['previous']
    test_cust['total_contacts'] = test_cust['campaign'] + test_cust['previous']
    
    # Campaign efficiency (duration per contact)
    train_cust['duration_per_contact'] = train_cust['duration'] / (train_cust['campaign'] + 1)  # +1 to avoid division by zero
    test_cust['duration_per_contact'] = test_cust['duration'] / (test_cust['campaign'] + 1)
    
    # 5. Customer lifecycle stage
    def get_lifecycle_stage(age, marital, job):
        if age <= 25:
            return 'student_young'
        elif age <= 35 and marital == 'single':
            return 'young_professional'
        elif age <= 45 and marital == 'married':
            return 'family_building'
        elif age <= 60:
            return 'career_peak'
        else:
            return 'pre_retirement'
    
    train_cust['lifecycle_stage'] = train_cust.apply(
        lambda x: get_lifecycle_stage(x['age'], x['marital'], x['job']), axis=1
    )
    test_cust['lifecycle_stage'] = test_cust.apply(
        lambda x: get_lifecycle_stage(x['age'], x['marital'], x['job']), axis=1
    )
    
    # 6. Communication effectiveness
    # Average duration suggests how engaged the customer is
    train_cust['communication_quality'] = np.where(
        train_cust['duration'] > train_cust['duration'].median(), 'high_engagement', 'low_engagement'
    )
    test_cust['communication_quality'] = np.where(
        test_cust['duration'] > train_cust['duration'].median(), 'high_engagement', 'low_engagement'  # Use train median for test
    )
    
    # 7. Risk profile
    def calculate_risk_profile(row):
        risk_score = 0
        if row['default'] == 'yes':
            risk_score += 3
        if row['loan'] == 'yes':
            risk_score += 1
        if row['balance'] < 0:
            risk_score += 2
        if row['job'] == 'unknown':
            risk_score += 1
        
        if risk_score <= 1:
            return 'low_risk'
        elif risk_score <= 3:
            return 'medium_risk'
        else:
            return 'high_risk'
    
    train_cust['risk_profile'] = train_cust.apply(calculate_risk_profile, axis=1)
    test_cust['risk_profile'] = test_cust.apply(calculate_risk_profile, axis=1)
    
    # 8. Contact recency feature (for pdays)
    def categorize_recency(pdays):
        if pdays == -1:
            return 'never_contacted'
        elif pdays <= 30:
            return 'recent'
        elif pdays <= 180:
            return 'medium_recency'
        else:
            return 'long_ago'
    
    train_cust['contact_recency'] = train_cust['pdays'].apply(categorize_recency)
    test_cust['contact_recency'] = test_cust['pdays'].apply(categorize_recency)
    
    print("Created customer segmentation features:")
    print("- age_group: Age-based categories")
    print("- balance_category: Balance-based categories")
    print("- financial_stability: Composite financial score (0-5)")
    print("- total_contacts: Sum of current and previous campaign contacts")
    print("- duration_per_contact: Average call duration efficiency")
    print("- lifecycle_stage: Customer life stage based on age/marital status")
    print("- communication_quality: Engagement level based on call duration")
    print("- risk_profile: Risk assessment based on financial indicators")
    print("- contact_recency: Recency of previous campaign contact")
    
    return train_cust, test_cust

# Apply customer feature engineering
X_train_processed, X_test_processed = create_customer_features(X_train_processed, X_test_processed)

print(f"\nSHAPE AFTER CUSTOMER FEATURE ENGINEERING:")
print(f"Training set: {X_train_processed.shape}")
print(f"Test set: {X_test_processed.shape}")

# Show some of the new categorical features
print(f"\nCUSTOMER SEGMENTATION DISTRIBUTIONS:")
print("Age group distribution:")
print(X_train_processed['age_group'].value_counts())
print("\nBalance category distribution:")
print(X_train_processed['balance_category'].value_counts())
print("\nFinancial stability distribution:")
print(X_train_processed['financial_stability'].value_counts())
print("\nRisk profile distribution:")
print(X_train_processed['risk_profile'].value_counts())

## Interaction Features

Based on our EDA insights about the most predictive features, we will create meaningful interaction features that capture complex relationships between variables.

In [ ]:
def create_interaction_features(df_train, df_test):
    """
    Create interaction features based on domain knowledge and EDA insights
    """
    train_inter = df_train.copy()
    test_inter = df_test.copy()
    
    print("CREATING INTERACTION FEATURES")
    print("="*50)
    
    # 1. Age and Job interaction (career stage impact)
    train_inter['age_job'] = train_inter['age_group'] + '_' + train_inter['job']
    test_inter['age_job'] = test_inter['age_group'] + '_' + test_inter['job']
    
    # 2. Balance and Housing loan interaction (financial capacity)
    train_inter['balance_housing'] = train_inter['balance_category'] + '_' + train_inter['housing']
    test_inter['balance_housing'] = test_inter['balance_category'] + '_' + test_inter['housing']
    
    # 3. Previous outcome and contact method interaction
    train_inter['poutcome_contact'] = train_inter['poutcome'] + '_' + train_inter['contact']
    test_inter['poutcome_contact'] = test_inter['poutcome'] + '_' + test_inter['contact']
    
    # 4. Duration and campaign intensity interaction
    train_inter['duration_campaign_ratio'] = train_inter['duration'] / (train_inter['campaign'] + 1)
    test_inter['duration_campaign_ratio'] = test_inter['duration'] / (test_inter['campaign'] + 1)
    
    # 5. Age and balance interaction (wealth accumulation stage)
    train_inter['age_balance_ratio'] = train_inter['age'] / (train_inter['balance_abs'] + 1)
    test_inter['age_balance_ratio'] = test_inter['age'] / (test_inter['balance_abs'] + 1)
    
    # 6. Education and job match (professional alignment)
    def education_job_match(education, job):
        high_education = ['tertiary']
        professional_jobs = ['management', 'technician', 'admin.']
        
        if education in high_education and job in professional_jobs:
            return 'high_match'
        elif education == 'secondary' and job in ['services', 'blue-collar']:
            return 'medium_match'
        else:
            return 'low_match'
    
    train_inter['education_job_match'] = train_inter.apply(
        lambda x: education_job_match(x['education'], x['job']), axis=1
    )
    test_inter['education_job_match'] = test_inter.apply(
        lambda x: education_job_match(x['education'], x['job']), axis=1
    )
    
    # 7. Seasonal and demographic interaction
    train_inter['season_age'] = train_inter['season'] + '_' + train_inter['age_group']
    test_inter['season_age'] = test_inter['season'] + '_' + test_inter['age_group']
    
    # 8. Campaign timing and previous outcome
    train_inter['month_poutcome'] = train_inter['month'] + '_' + train_inter['poutcome']
    test_inter['month_poutcome'] = test_inter['month'] + '_' + test_inter['poutcome']
    
    # 9. Financial stability and contact quality
    train_inter['stability_communication'] = (
        train_inter['financial_stability'].astype(str) + '_' + train_inter['communication_quality']
    )
    test_inter['stability_communication'] = (
        test_inter['financial_stability'].astype(str) + '_' + test_inter['communication_quality']
    )
    
    # 10. Marital status and housing interaction (family financial status)
    train_inter['marital_housing'] = train_inter['marital'] + '_' + train_inter['housing']
    test_inter['marital_housing'] = test_inter['marital'] + '_' + test_inter['housing']
    
    print("Created interaction features:")
    print("- age_job: Age group and job type combination")
    print("- balance_housing: Balance category and housing loan status")
    print("- poutcome_contact: Previous outcome and contact method")
    print("- duration_campaign_ratio: Call efficiency per campaign contact")
    print("- age_balance_ratio: Age to balance ratio (wealth accumulation)")
    print("- education_job_match: Education and job professional alignment")
    print("- season_age: Seasonal and age group interaction")
    print("- month_poutcome: Campaign month and previous outcome")
    print("- stability_communication: Financial stability and communication quality")
    print("- marital_housing: Marital status and housing loan interaction")
    
    return train_inter, test_inter

# Apply interaction feature engineering
X_train_processed, X_test_processed = create_interaction_features(X_train_processed, X_test_processed)

print(f"\nSHAPE AFTER INTERACTION FEATURE ENGINEERING:")
print(f"Training set: {X_train_processed.shape}")
print(f"Test set: {X_test_processed.shape}")

# Show total number of features created
total_original = len(original_features)
total_current = X_train_processed.shape[1] - 1  # Subtract 1 for ID column
features_added = total_current - total_original

print(f"\nFEATURE ENGINEERING SUMMARY:")
print(f"Original features: {total_original}")
print(f"Current features: {total_current}")
print(f"Features added: {features_added}")
print(f"Feature expansion ratio: {total_current/total_original:.2f}x")

# Categorical Encoding and Numerical Scaling

With our comprehensive feature engineering complete, we now need to prepare our features for machine learning algorithms. This involves encoding categorical variables and scaling numerical features appropriately.

## Encoding Strategy

Based on our EDA insights, we will use:
1. **Target Encoding** for high-cardinality categorical features
2. **One-Hot Encoding** for low-cardinality categorical features  
3. **Standard Scaling** for numerical features
4. **Robust Scaling** for features with outliers

In [ ]:
# Identify categorical and numerical features in processed dataset
print("FEATURE TYPE IDENTIFICATION")
print("="*60)

# Get current feature types
categorical_features_processed = X_train_processed.select_dtypes(include=['object']).columns.tolist()
numerical_features_processed = X_train_processed.select_dtypes(include=[np.number]).columns.tolist()

# Remove ID from numerical features
if 'id' in numerical_features_processed:
    numerical_features_processed.remove('id')

print(f"Categorical features ({len(categorical_features_processed)}):")
for i, feature in enumerate(categorical_features_processed, 1):
    unique_count = X_train_processed[feature].nunique()
    print(f"  {i:2}. {feature:25} ({unique_count:3} unique values)")

print(f"\nNumerical features ({len(numerical_features_processed)}):")
for i, feature in enumerate(numerical_features_processed, 1):
    print(f"  {i:2}. {feature:25}")

# Categorize features by cardinality for encoding strategy
low_cardinality = []  # <= 10 unique values - One-hot encode
high_cardinality = []  # > 10 unique values - Target encode

for feature in categorical_features_processed:
    unique_count = X_train_processed[feature].nunique()
    if unique_count <= 10:
        low_cardinality.append(feature)
    else:
        high_cardinality.append(feature)

print(f"\n" + "="*60)
print("ENCODING STRATEGY BY CARDINALITY")
print("="*60)
print(f"Low cardinality features (One-Hot Encoding): {len(low_cardinality)}")
for feature in low_cardinality:
    unique_count = X_train_processed[feature].nunique()
    print(f"  - {feature}: {unique_count} categories")

print(f"\nHigh cardinality features (Target Encoding): {len(high_cardinality)}")
for feature in high_cardinality:
    unique_count = X_train_processed[feature].nunique()
    print(f"  - {feature}: {unique_count} categories")

In [ ]:
# Target Encoding for High-Cardinality Features
from sklearn.model_selection import KFold

def target_encode_features(X_train, y_train, X_test, categorical_cols, cv_folds=5, smoothing=1.0):
    """
    Perform target encoding with cross-validation to prevent overfitting
    """
    X_train_encoded = X_train.copy()
    X_test_encoded = X_test.copy()
    
    # Global target mean for smoothing
    global_mean = y_train.mean()
    
    print("APPLYING TARGET ENCODING")
    print("="*50)
    
    # Initialize KFold for cross-validation
    kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    for feature in categorical_cols:
        print(f"Encoding {feature}...")
        
        # Initialize encoded feature with global mean
        train_encoded = np.full(len(X_train), global_mean)
        
        # Cross-validation encoding for training set
        for train_idx, val_idx in kf.split(X_train):
            # Calculate target means for this fold
            target_means = y_train.iloc[train_idx].groupby(X_train[feature].iloc[train_idx]).mean()
            target_counts = y_train.iloc[train_idx].groupby(X_train[feature].iloc[train_idx]).count()
            
            # Apply smoothing: (count * mean + smoothing * global_mean) / (count + smoothing)
            smoothed_means = (target_counts * target_means + smoothing * global_mean) / (target_counts + smoothing)
            
            # Map to validation set
            train_encoded[val_idx] = X_train[feature].iloc[val_idx].map(smoothed_means).fillna(global_mean)
        
        # For test set, use all training data
        target_means = y_train.groupby(X_train[feature]).mean()
        target_counts = y_train.groupby(X_train[feature]).count()
        smoothed_means = (target_counts * target_means + smoothing * global_mean) / (target_counts + smoothing)
        
        test_encoded = X_test[feature].map(smoothed_means).fillna(global_mean)
        
        # Store encoded features
        X_train_encoded[f'{feature}_target_encoded'] = train_encoded
        X_test_encoded[f'{feature}_target_encoded'] = test_encoded
        
        # Show encoding statistics
        print(f"  - Original categories: {X_train[feature].nunique()}")
        print(f"  - Encoded range: [{train_encoded.min():.4f}, {train_encoded.max():.4f}]")
        print(f"  - Correlation with target: {np.corrcoef(train_encoded, y_train)[0,1]:.4f}")
    
    return X_train_encoded, X_test_encoded

# Apply target encoding to high-cardinality features
if high_cardinality:
    X_train_target_encoded, X_test_target_encoded = target_encode_features(
        X_train_processed, y_train, X_test_processed, high_cardinality
    )
    print(f"\nTarget encoding completed for {len(high_cardinality)} features")
else:
    X_train_target_encoded = X_train_processed.copy()
    X_test_target_encoded = X_test_processed.copy()
    print("No high-cardinality features found for target encoding")

In [ ]:
# One-Hot Encoding for Low-Cardinality Features
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

def one_hot_encode_features(X_train, X_test, categorical_cols):
    """
    Apply one-hot encoding to low-cardinality categorical features
    """
    if not categorical_cols:
        return X_train.copy(), X_test.copy()
    
    print("APPLYING ONE-HOT ENCODING")
    print("="*50)
    
    X_train_encoded = X_train.copy()
    X_test_encoded = X_test.copy()
    
    # Initialize OneHotEncoder
    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    
    for feature in categorical_cols:
        print(f"One-hot encoding {feature}...")
        
        # Fit on training data and transform both sets
        train_feature = X_train[[feature]]
        test_feature = X_test[[feature]]
        
        # Fit and transform
        train_encoded = encoder.fit_transform(train_feature)
        test_encoded = encoder.transform(test_feature)
        
        # Get feature names
        feature_names = encoder.get_feature_names_out([feature])
        
        # Create DataFrames with proper column names
        train_encoded_df = pd.DataFrame(train_encoded, columns=feature_names, index=X_train.index)
        test_encoded_df = pd.DataFrame(test_encoded, columns=feature_names, index=X_test.index)
        
        # Add to the main dataframes
        X_train_encoded = pd.concat([X_train_encoded, train_encoded_df], axis=1)
        X_test_encoded = pd.concat([X_test_encoded, test_encoded_df], axis=1)
        
        print(f"  - Original categories: {X_train[feature].nunique()}")
        print(f"  - New binary features created: {len(feature_names)}")
        print(f"  - Feature names: {list(feature_names)}")
    
    print(f"\nShape after one-hot encoding:")
    print(f"  - Training: {X_train_encoded.shape}")
    print(f"  - Test: {X_test_encoded.shape}")
    
    return X_train_encoded, X_test_encoded

# Apply one-hot encoding to low-cardinality features
if low_cardinality:
    X_train_onehot, X_test_onehot = one_hot_encode_features(
        X_train_target_encoded, X_test_target_encoded, low_cardinality
    )
    print(f"\nOne-hot encoding completed for {len(low_cardinality)} features")
else:
    X_train_onehot = X_train_target_encoded.copy()
    X_test_onehot = X_test_target_encoded.copy()
    print("No low-cardinality features found for one-hot encoding")

In [ ]:
# Numerical Feature Scaling
from sklearn.preprocessing import StandardScaler, RobustScaler

def scale_numerical_features(X_train, X_test, numerical_cols, scaling_method='standard'):
    """
    Scale numerical features using specified method
    """
    print("SCALING NUMERICAL FEATURES")
    print("="*50)
    
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    
    if scaling_method == 'standard':
        scaler = StandardScaler()
        print("Using StandardScaler (mean=0, std=1)")
    elif scaling_method == 'robust':
        scaler = RobustScaler()
        print("Using RobustScaler (median=0, IQR=1)")
    else:
        raise ValueError("scaling_method must be 'standard' or 'robust'")
    
    print(f"Scaling {len(numerical_cols)} numerical features...")
    
    # Fit scaler on training data and transform both sets
    X_train_scaled[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
    X_test_scaled[numerical_cols] = scaler.transform(X_test[numerical_cols])
    
    # Show scaling statistics
    print(f"\nScaling statistics:")
    for feature in numerical_cols[:5]:  # Show first 5 features
        original_mean = X_train[feature].mean()
        original_std = X_train[feature].std()
        scaled_mean = X_train_scaled[feature].mean()
        scaled_std = X_train_scaled[feature].std()
        
        print(f"  {feature:20}: Original(μ={original_mean:6.2f}, σ={original_std:6.2f}) → "
              f"Scaled(μ={scaled_mean:6.3f}, σ={scaled_std:6.3f})")
    
    if len(numerical_cols) > 5:
        print(f"  ... and {len(numerical_cols) - 5} more features")
    
    return X_train_scaled, X_test_scaled, scaler

# Identify which features to scale (exclude ID and already encoded features)
features_to_scale = [col for col in numerical_features_processed 
                    if col not in ['id'] and not col.endswith('_target_encoded')]

print(f"Features to scale ({len(features_to_scale)}):")
for feature in features_to_scale:
    print(f"  - {feature}")

# Apply scaling (use robust scaler due to outliers identified in EDA)
X_train_scaled, X_test_scaled, numerical_scaler = scale_numerical_features(
    X_train_onehot, X_test_onehot, features_to_scale, scaling_method='robust'
)

print(f"\nFinal dataset shapes after all preprocessing:")
print(f"  - Training: {X_train_scaled.shape}")
print(f"  - Test: {X_test_scaled.shape}")
print(f"  - Target: {y_train.shape}")

## Feature Selection and Final Preparation

Before model training, we need to:
1. Remove original categorical features that have been encoded
2. Handle any remaining high-correlation features
3. Prepare final feature sets for modeling
4. Create train/validation splits

In [ ]:
# Final Feature Selection and Dataset Preparation
def prepare_final_datasets(X_train, X_test, categorical_features):
    """
    Prepare final datasets by removing original categorical features and 
    handling any remaining issues
    """
    print("FINAL DATASET PREPARATION")
    print("="*50)
    
    # Create copies
    X_train_final = X_train.copy()
    X_test_final = X_test.copy()
    
    # Remove original categorical features (keep only encoded versions)
    features_to_remove = []
    for feature in categorical_features:
        if feature in X_train_final.columns:
            features_to_remove.append(feature)
    
    print(f"Removing {len(features_to_remove)} original categorical features:")
    for feature in features_to_remove:
        print(f"  - {feature}")
    
    X_train_final = X_train_final.drop(columns=features_to_remove)
    X_test_final = X_test_final.drop(columns=features_to_remove)
    
    # Remove ID column if present
    if 'id' in X_train_final.columns:
        X_train_final = X_train_final.drop(columns=['id'])
    if 'id' in X_test_final.columns:
        test_ids = X_test_final['id'].copy()  # Save for submission
        X_test_final = X_test_final.drop(columns=['id'])
    else:
        test_ids = None
    
    # Ensure same columns in both datasets
    train_cols = set(X_train_final.columns)
    test_cols = set(X_test_final.columns)
    
    if train_cols != test_cols:
        print(f"\nColumn mismatch detected:")
        print(f"  - Training only: {train_cols - test_cols}")
        print(f"  - Test only: {test_cols - train_cols}")
        
        # Keep only common columns
        common_cols = train_cols.intersection(test_cols)
        X_train_final = X_train_final[list(common_cols)]
        X_test_final = X_test_final[list(common_cols)]
        print(f"  - Using {len(common_cols)} common columns")
    
    # Check for any remaining missing values
    train_missing = X_train_final.isnull().sum().sum()
    test_missing = X_test_final.isnull().sum().sum()
    
    if train_missing > 0 or test_missing > 0:
        print(f"\nWarning: Found missing values - Train: {train_missing}, Test: {test_missing}")
        # Fill with median for numerical, mode for categorical
        for col in X_train_final.columns:
            if X_train_final[col].dtype in ['float64', 'int64']:
                fill_value = X_train_final[col].median()
                X_train_final[col].fillna(fill_value, inplace=True)
                X_test_final[col].fillna(fill_value, inplace=True)
            else:
                fill_value = X_train_final[col].mode()[0] if len(X_train_final[col].mode()) > 0 else 'unknown'
                X_train_final[col].fillna(fill_value, inplace=True)
                X_test_final[col].fillna(fill_value, inplace=True)
    
    print(f"\nFinal dataset shapes:")
    print(f"  - Training features: {X_train_final.shape}")
    print(f"  - Test features: {X_test_final.shape}")
    print(f"  - Feature count: {X_train_final.shape[1]}")
    
    return X_train_final, X_test_final, test_ids

# Prepare final datasets
X_train_final, X_test_final, test_ids = prepare_final_datasets(
    X_train_scaled, X_test_scaled, categorical_features_processed
)

# Show final feature list
print(f"\nFinal feature list ({len(X_train_final.columns)} features):")
for i, feature in enumerate(X_train_final.columns, 1):
    print(f"  {i:2}. {feature}")

# Quick correlation check for multicollinearity
print(f"\n" + "="*50)
print("MULTICOLLINEARITY CHECK")
print("="*50)

# Calculate correlation matrix for final features
final_corr_matrix = X_train_final.corr()

# Find highly correlated pairs (> 0.9)
high_corr_threshold = 0.9
high_corr_pairs = []

for i in range(len(final_corr_matrix.columns)):
    for j in range(i+1, len(final_corr_matrix.columns)):
        corr_value = abs(final_corr_matrix.iloc[i, j])
        if corr_value > high_corr_threshold:
            high_corr_pairs.append({
                'Feature_1': final_corr_matrix.columns[i],
                'Feature_2': final_corr_matrix.columns[j],
                'Correlation': final_corr_matrix.iloc[i, j]
            })

if high_corr_pairs:
    print(f"Found {len(high_corr_pairs)} highly correlated feature pairs (|r| > {high_corr_threshold}):")
    for pair in high_corr_pairs:
        print(f"  - {pair['Feature_1']} ↔ {pair['Feature_2']}: {pair['Correlation']:.3f}")
else:
    print(f"No highly correlated feature pairs found (threshold: {high_corr_threshold})")

print(f"\nDataset ready for modeling!")
print(f"Original features: {len(original_features)}")
print(f"Final features: {X_train_final.shape[1]}")
print(f"Feature expansion: {X_train_final.shape[1]/len(original_features):.1f}x")

# Model Development and Training

With our comprehensive feature engineering and preprocessing complete, we now move to the core machine learning phase. Our approach will focus on building robust, high-performing models optimized for this binary classification task.

## Modeling Strategy

Based on our EDA insights and the nature of this tabular data problem, our strategy includes:

1. **Gradient Boosting Models** - LightGBM and XGBoost (proven winners in tabular competitions)
2. **Proper Cross-Validation** - Stratified CV to handle class imbalance
3. **Hyperparameter Optimization** - Systematic tuning for optimal performance
4. **Model Ensemble** - Combining diverse models for improved generalization
5. **Comprehensive Evaluation** - Multiple metrics suited for imbalanced classification

In [ ]:
# Cross-Validation Setup and Evaluation Framework
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score, 
    f1_score, classification_report, confusion_matrix, roc_curve, precision_recall_curve
)
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Set up cross-validation strategy
def setup_cross_validation(n_splits=5, random_state=42):
    """
    Set up stratified cross-validation for imbalanced dataset
    """
    print("CROSS-VALIDATION SETUP")
    print("="*50)
    
    # Use stratified CV to maintain class distribution
    cv_strategy = StratifiedKFold(
        n_splits=n_splits, 
        shuffle=True, 
        random_state=random_state
    )
    
    print(f"Strategy: {n_splits}-Fold Stratified Cross-Validation")
    print(f"Shuffle: True, Random State: {random_state}")
    
    # Verify stratification
    fold_distributions = []
    for fold, (train_idx, val_idx) in enumerate(cv_strategy.split(X_train_final, y_train)):
        train_dist = y_train.iloc[train_idx].mean()
        val_dist = y_train.iloc[val_idx].mean()
        fold_distributions.append({
            'Fold': fold + 1,
            'Train_Size': len(train_idx),
            'Val_Size': len(val_idx),
            'Train_Positive_%': f"{train_dist:.3f}",
            'Val_Positive_%': f"{val_dist:.3f}"
        })
    
    fold_df = pd.DataFrame(fold_distributions)
    print(f"\nFold Distribution Verification:")
    print(fold_df)
    
    return cv_strategy

# Comprehensive evaluation metrics
def evaluate_model(y_true, y_pred, y_pred_proba=None, model_name="Model"):
    """
    Comprehensive model evaluation with multiple metrics
    """
    print(f"\n{model_name.upper()} EVALUATION")
    print("="*50)
    
    # Basic metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    
    # AUC if probabilities provided
    if y_pred_proba is not None:
        auc = roc_auc_score(y_true, y_pred_proba)
        print(f"ROC-AUC:   {auc:.4f}")
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    print(f"\nConfusion Matrix:")
    print(f"TN: {cm[0,0]:,}  FP: {cm[0,1]:,}")
    print(f"FN: {cm[1,0]:,}  TP: {cm[1,1]:,}")
    
    # Classification Report
    print(f"\nDetailed Classification Report:")
    print(classification_report(y_true, y_pred))
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': roc_auc_score(y_true, y_pred_proba) if y_pred_proba is not None else None,
        'confusion_matrix': cm
    }

# Setup cross-validation
cv_strategy = setup_cross_validation(n_splits=5)

print(f"\nDataset ready for model training:")
print(f"Features: {X_train_final.shape[1]}")
print(f"Training samples: {X_train_final.shape[0]:,}")
print(f"Class distribution: {y_train.value_counts().to_dict()}")

## LightGBM Model - Primary Gradient Boosting Approach

LightGBM is our primary model choice due to its excellent performance on tabular data, efficient memory usage, and built-in handling of categorical features and class imbalance.

In [ ]:
# LightGBM Model Training and Evaluation
def train_lightgbm_model(X_train, y_train, cv_strategy, params=None):
    """
    Train LightGBM model with cross-validation and return results
    """
    print("TRAINING LIGHTGBM MODEL")
    print("="*50)
    
    # Default parameters optimized for binary classification
    if params is None:
        params = {
            'objective': 'binary',
            'metric': 'auc',
            'boosting_type': 'gbdt',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': -1,
            'random_state': 42,
            'is_unbalance': True  # Handle class imbalance
        }
    
    print("Model Parameters:")
    for key, value in params.items():
        print(f"  {key}: {value}")
    
    # Cross-validation training
    cv_scores = []
    cv_predictions = np.zeros(len(X_train))
    cv_predictions_proba = np.zeros(len(X_train))
    feature_importance = np.zeros(X_train.shape[1])
    
    models = []
    
    for fold, (train_idx, val_idx) in enumerate(cv_strategy.split(X_train, y_train)):
        print(f"\nTraining Fold {fold + 1}...")
        
        # Split data
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # Create LightGBM datasets
        train_data = lgb.Dataset(X_fold_train, label=y_fold_train)
        val_data = lgb.Dataset(X_fold_val, label=y_fold_val, reference=train_data)
        
        # Train model
        model = lgb.train(
            params,
            train_data,
            valid_sets=[train_data, val_data],
            valid_names=['train', 'val'],
            num_boost_round=1000,
            callbacks=[
                lgb.early_stopping(stopping_rounds=100, verbose=False),
                lgb.log_evaluation(period=0)  # Suppress detailed logging
            ]
        )
        
        # Predictions
        val_pred_proba = model.predict(X_fold_val, num_iteration=model.best_iteration)
        val_pred = (val_pred_proba > 0.5).astype(int)
        
        # Store predictions
        cv_predictions[val_idx] = val_pred
        cv_predictions_proba[val_idx] = val_pred_proba
        
        # Calculate fold AUC
        fold_auc = roc_auc_score(y_fold_val, val_pred_proba)
        cv_scores.append(fold_auc)
        
        # Accumulate feature importance
        feature_importance += model.feature_importance(importance_type='gain')
        
        # Store model
        models.append(model)
        
        print(f"  Fold {fold + 1} AUC: {fold_auc:.4f}")
    
    # Average feature importance
    feature_importance /= len(models)
    
    # Overall CV performance
    overall_auc = roc_auc_score(y_train, cv_predictions_proba)
    mean_cv_auc = np.mean(cv_scores)
    std_cv_auc = np.std(cv_scores)
    
    print(f"\n" + "="*50)
    print("CROSS-VALIDATION RESULTS")
    print("="*50)
    print(f"Individual Fold AUCs: {[f'{score:.4f}' for score in cv_scores]}")
    print(f"Mean CV AUC: {mean_cv_auc:.4f} ± {std_cv_auc:.4f}")
    print(f"Overall AUC: {overall_auc:.4f}")
    
    return {
        'models': models,
        'cv_predictions': cv_predictions,
        'cv_predictions_proba': cv_predictions_proba,
        'cv_scores': cv_scores,
        'mean_auc': mean_cv_auc,
        'std_auc': std_cv_auc,
        'overall_auc': overall_auc,
        'feature_importance': feature_importance,
        'feature_names': X_train.columns.tolist()
    }

# Train LightGBM model
lgb_results = train_lightgbm_model(X_train_final, y_train, cv_strategy)

# Evaluate LightGBM predictions
lgb_evaluation = evaluate_model(
    y_train, 
    lgb_results['cv_predictions'], 
    lgb_results['cv_predictions_proba'],
    "LightGBM Cross-Validation"
)

## XGBoost Model - Alternative Gradient Boosting

XGBoost provides a different gradient boosting implementation that often complements LightGBM well in ensemble approaches. We'll train it with similar methodology for comparison.

In [ ]:
# XGBoost Model Training and Evaluation
def train_xgboost_model(X_train, y_train, cv_strategy, params=None):
    """
    Train XGBoost model with cross-validation and return results
    """
    print("TRAINING XGBOOST MODEL")
    print("="*50)
    
    # Default parameters optimized for binary classification
    if params is None:
        params = {
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'booster': 'gbtree',
            'max_depth': 6,
            'learning_rate': 0.05,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42,
            'n_jobs': -1,
            'verbosity': 0,
            'early_stopping_rounds': 100  # Move early stopping to constructor
        }
    
    print("Model Parameters:")
    for key, value in params.items():
        print(f"  {key}: {value}")
    
    # Calculate scale_pos_weight for class imbalance
    scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
    params['scale_pos_weight'] = scale_pos_weight
    print(f"  scale_pos_weight: {scale_pos_weight:.2f} (for class imbalance)")
    
    # Cross-validation training
    cv_scores = []
    cv_predictions = np.zeros(len(X_train))
    cv_predictions_proba = np.zeros(len(X_train))
    feature_importance = np.zeros(X_train.shape[1])
    
    models = []
    
    for fold, (train_idx, val_idx) in enumerate(cv_strategy.split(X_train, y_train)):
        print(f"\nTraining Fold {fold + 1}...")
        
        # Split data
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # Create XGBoost model
        model = xgb.XGBClassifier(**params)
        
        # Train model with validation set
        model.fit(
            X_fold_train, y_fold_train,
            eval_set=[(X_fold_train, y_fold_train), (X_fold_val, y_fold_val)],
            verbose=False
        )
        
        # Predictions
        val_pred_proba = model.predict_proba(X_fold_val)[:, 1]
        val_pred = model.predict(X_fold_val)
        
        # Store predictions
        cv_predictions[val_idx] = val_pred
        cv_predictions_proba[val_idx] = val_pred_proba
        
        # Calculate fold AUC
        fold_auc = roc_auc_score(y_fold_val, val_pred_proba)
        cv_scores.append(fold_auc)
        
        # Accumulate feature importance
        feature_importance += model.feature_importances_
        
        # Store model
        models.append(model)
        
        print(f"  Fold {fold + 1} AUC: {fold_auc:.4f}")
    
    # Average feature importance
    feature_importance /= len(models)
    
    # Overall CV performance
    overall_auc = roc_auc_score(y_train, cv_predictions_proba)
    mean_cv_auc = np.mean(cv_scores)
    std_cv_auc = np.std(cv_scores)
    
    print(f"\n" + "="*50)
    print("CROSS-VALIDATION RESULTS")
    print("="*50)
    print(f"Individual Fold AUCs: {[f'{score:.4f}' for score in cv_scores]}")
    print(f"Mean CV AUC: {mean_cv_auc:.4f} ± {std_cv_auc:.4f}")
    print(f"Overall AUC: {overall_auc:.4f}")
    
    return {
        'models': models,
        'cv_predictions': cv_predictions,
        'cv_predictions_proba': cv_predictions_proba,
        'cv_scores': cv_scores,
        'mean_auc': mean_cv_auc,
        'std_auc': std_cv_auc,
        'overall_auc': overall_auc,
        'feature_importance': feature_importance,
        'feature_names': X_train.columns.tolist()
    }

# Train XGBoost model
xgb_results = train_xgboost_model(X_train_final, y_train, cv_strategy)

# Evaluate XGBoost predictions
xgb_evaluation = evaluate_model(
    y_train, 
    xgb_results['cv_predictions'], 
    xgb_results['cv_predictions_proba'],
    "XGBoost Cross-Validation"
)

In [ ]:
# Feature Importance Analysis
def analyze_feature_importance(lgb_results, xgb_results, top_n=20):
    """
    Analyze and visualize feature importance from both models
    """
    print("FEATURE IMPORTANCE ANALYSIS")
    print("="*60)
    
    # Create importance DataFrames
    lgb_importance = pd.DataFrame({
        'feature': lgb_results['feature_names'],
        'lgb_importance': lgb_results['feature_importance']
    })
    
    xgb_importance = pd.DataFrame({
        'feature': xgb_results['feature_names'],
        'xgb_importance': xgb_results['feature_importance']
    })
    
    # Merge importance scores
    importance_df = pd.merge(lgb_importance, xgb_importance, on='feature')
    
    # Calculate average importance
    importance_df['avg_importance'] = (importance_df['lgb_importance'] + importance_df['xgb_importance']) / 2
    
    # Sort by average importance
    importance_df = importance_df.sort_values('avg_importance', ascending=False)
    
    print(f"Top {top_n} Most Important Features:")
    print("-" * 60)
    print(f"{'Rank':<4} {'Feature':<30} {'LGB':<8} {'XGB':<8} {'Avg':<8}")
    print("-" * 60)
    
    for i, row in importance_df.head(top_n).iterrows():
        print(f"{importance_df.index.get_loc(i)+1:<4} {row['feature']:<30} "
              f"{row['lgb_importance']:<8.0f} {row['xgb_importance']:<8.2f} {row['avg_importance']:<8.1f}")
    
    # Visualize feature importance
    fig, axes = plt.subplots(1, 3, figsize=(20, 8))
    
    # LightGBM importance
    top_lgb = importance_df.head(15)
    axes[0].barh(range(len(top_lgb)), top_lgb['lgb_importance'], color='lightblue')
    axes[0].set_yticks(range(len(top_lgb)))
    axes[0].set_yticklabels(top_lgb['feature'])
    axes[0].set_title('LightGBM Feature Importance', fontweight='bold')
    axes[0].set_xlabel('Importance Score')
    axes[0].invert_yaxis()
    
    # XGBoost importance
    top_xgb = importance_df.head(15)
    axes[1].barh(range(len(top_xgb)), top_xgb['xgb_importance'], color='lightcoral')
    axes[1].set_yticks(range(len(top_xgb)))
    axes[1].set_yticklabels(top_xgb['feature'])
    axes[1].set_title('XGBoost Feature Importance', fontweight='bold')
    axes[1].set_xlabel('Importance Score')
    axes[1].invert_yaxis()
    
    # Average importance
    top_avg = importance_df.head(15)
    axes[2].barh(range(len(top_avg)), top_avg['avg_importance'], color='lightgreen')
    axes[2].set_yticks(range(len(top_avg)))
    axes[2].set_yticklabels(top_avg['feature'])
    axes[2].set_title('Average Feature Importance', fontweight='bold')
    axes[2].set_xlabel('Average Importance Score')
    axes[2].invert_yaxis()
    
    plt.tight_layout()
    plt.show()
    
    return importance_df

# Analyze feature importance
feature_importance_df = analyze_feature_importance(lgb_results, xgb_results)

# Compare model performance
print(f"\n" + "="*60)
print("MODEL PERFORMANCE COMPARISON")
print("="*60)
print(f"{'Metric':<20} {'LightGBM':<12} {'XGBoost':<12}")
print("-" * 45)
print(f"{'Mean CV AUC':<20} {lgb_results['mean_auc']:<12.4f} {xgb_results['mean_auc']:<12.4f}")
print(f"{'CV AUC Std':<20} {lgb_results['std_auc']:<12.4f} {xgb_results['std_auc']:<12.4f}")
print(f"{'Overall AUC':<20} {lgb_results['overall_auc']:<12.4f} {xgb_results['overall_auc']:<12.4f}")

# Determine best model
if lgb_results['overall_auc'] > xgb_results['overall_auc']:
    best_model_name = "LightGBM"
    best_results = lgb_results
else:
    best_model_name = "XGBoost"
    best_results = xgb_results

print(f"\nBest performing model: {best_model_name} (AUC: {best_results['overall_auc']:.4f})")

## Ensemble Model - Combining Predictions

Ensemble methods often provide the best performance in Kaggle competitions by combining the strengths of different models. We'll create a weighted ensemble of our best models.

In [ ]:
# Ensemble Model Creation and Evaluation
def create_ensemble_predictions(lgb_results, xgb_results, weights=None):
    """
    Create ensemble predictions from multiple models
    """
    print("CREATING ENSEMBLE MODEL")
    print("="*50)
    
    if weights is None:
        # Weight by relative performance (AUC-based)
        lgb_weight = lgb_results['overall_auc']
        xgb_weight = xgb_results['overall_auc']
        total_weight = lgb_weight + xgb_weight
        weights = [lgb_weight / total_weight, xgb_weight / total_weight]
    
    print(f"Ensemble weights:")
    print(f"  LightGBM: {weights[0]:.3f}")
    print(f"  XGBoost:  {weights[1]:.3f}")
    
    # Weighted average of probabilities
    ensemble_proba = (
        weights[0] * lgb_results['cv_predictions_proba'] +
        weights[1] * xgb_results['cv_predictions_proba']
    )
    
    # Convert to binary predictions
    ensemble_pred = (ensemble_proba > 0.5).astype(int)
    
    # Calculate ensemble AUC
    ensemble_auc = roc_auc_score(y_train, ensemble_proba)
    
    print(f"\nEnsemble Performance:")
    print(f"  AUC: {ensemble_auc:.4f}")
    print(f"  LightGBM AUC: {lgb_results['overall_auc']:.4f}")
    print(f"  XGBoost AUC:  {xgb_results['overall_auc']:.4f}")
    
    improvement_lgb = ensemble_auc - lgb_results['overall_auc']
    improvement_xgb = ensemble_auc - xgb_results['overall_auc']
    
    print(f"\nImprovement over individual models:")
    print(f"  vs LightGBM: {improvement_lgb:+.4f}")
    print(f"  vs XGBoost:  {improvement_xgb:+.4f}")
    
    return {
        'predictions': ensemble_pred,
        'predictions_proba': ensemble_proba,
        'auc': ensemble_auc,
        'weights': weights
    }

# Create ensemble predictions
ensemble_results = create_ensemble_predictions(lgb_results, xgb_results)

# Evaluate ensemble model
ensemble_evaluation = evaluate_model(
    y_train,
    ensemble_results['predictions'],
    ensemble_results['predictions_proba'],
    "Ensemble Model"
)

# Visualize model comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# AUC comparison
models = ['LightGBM', 'XGBoost', 'Ensemble']
aucs = [lgb_results['overall_auc'], xgb_results['overall_auc'], ensemble_results['auc']]
colors = ['lightblue', 'lightcoral', 'lightgreen']

bars = axes[0].bar(models, aucs, color=colors, alpha=0.7)
axes[0].set_title('Model AUC Comparison', fontweight='bold')
axes[0].set_ylabel('AUC Score')
axes[0].set_ylim(0.8, max(aucs) + 0.01)
axes[0].grid(True, alpha=0.3)

# Add value labels on bars
for bar, auc in zip(bars, aucs):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
                f'{auc:.4f}', ha='center', va='bottom', fontweight='bold')

# ROC Curves
from sklearn.metrics import roc_curve

# Calculate ROC curves
lgb_fpr, lgb_tpr, _ = roc_curve(y_train, lgb_results['cv_predictions_proba'])
xgb_fpr, xgb_tpr, _ = roc_curve(y_train, xgb_results['cv_predictions_proba'])
ens_fpr, ens_tpr, _ = roc_curve(y_train, ensemble_results['predictions_proba'])

axes[1].plot(lgb_fpr, lgb_tpr, label=f'LightGBM (AUC = {lgb_results["overall_auc"]:.4f})', color='blue')
axes[1].plot(xgb_fpr, xgb_tpr, label=f'XGBoost (AUC = {xgb_results["overall_auc"]:.4f})', color='red')
axes[1].plot(ens_fpr, ens_tpr, label=f'Ensemble (AUC = {ensemble_results["auc"]:.4f})', color='green', linewidth=2)
axes[1].plot([0, 1], [0, 1], 'k--', alpha=0.5)
axes[1].set_title('ROC Curves Comparison', fontweight='bold')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Final model selection
print(f"\n" + "="*60)
print("FINAL MODEL SELECTION")
print("="*60)

final_models = {
    'LightGBM': lgb_results,
    'XGBoost': xgb_results,
    'Ensemble': ensemble_results
}

best_auc = 0
best_model = None
for name, results in final_models.items():
    auc = results['auc'] if 'auc' in results else results['overall_auc']
    print(f"{name:<10}: AUC = {auc:.4f}")
    if auc > best_auc:
        best_auc = auc
        best_model = name

print(f"\nBest model for final predictions: {best_model} (AUC: {best_auc:.4f})")

# Store best model results for prediction
if best_model == 'Ensemble':
    final_model_results = ensemble_results
    final_model_results['models'] = {
        'lgb': lgb_results['models'],
        'xgb': xgb_results['models'],
        'weights': ensemble_results['weights']
    }
elif best_model == 'LightGBM':
    final_model_results = lgb_results
else:
    final_model_results = xgb_results

print(f"\nModel ready for test set predictions!")

## Test Set Predictions and Submission

Now we'll use our best-performing model to make predictions on the test set and prepare the final submission file for Kaggle.

In [ ]:
# Test Set Predictions and Submission File Creation
def generate_test_predictions(models_dict, X_test, model_type='ensemble'):
    """
    Generate predictions on test set using trained models
    """
    print("GENERATING TEST SET PREDICTIONS")
    print("="*50)
    
    if model_type == 'ensemble':
        print("Using ensemble of LightGBM and XGBoost models")
        
        # Get models and weights
        lgb_models = models_dict['models']['lgb']
        xgb_models = models_dict['models']['xgb']
        weights = models_dict['models']['weights']
        
        # Average predictions across folds for each model
        lgb_test_preds = np.zeros(len(X_test))
        xgb_test_preds = np.zeros(len(X_test))
        
        # LightGBM predictions
        for model in lgb_models:
            lgb_test_preds += model.predict(X_test, num_iteration=model.best_iteration)
        lgb_test_preds /= len(lgb_models)
        
        # XGBoost predictions
        for model in xgb_models:
            xgb_test_preds += model.predict_proba(X_test)[:, 1]
        xgb_test_preds /= len(xgb_models)
        
        # Ensemble predictions
        test_predictions = weights[0] * lgb_test_preds + weights[1] * xgb_test_preds
        
        print(f"  LightGBM average prediction: {lgb_test_preds.mean():.4f}")
        print(f"  XGBoost average prediction:  {xgb_test_preds.mean():.4f}")
        print(f"  Ensemble average prediction: {test_predictions.mean():.4f}")
        
    elif model_type == 'lightgbm':
        print("Using LightGBM models")
        models = models_dict['models']
        test_predictions = np.zeros(len(X_test))
        
        for model in models:
            test_predictions += model.predict(X_test, num_iteration=model.best_iteration)
        test_predictions /= len(models)
        
        print(f"  Average prediction: {test_predictions.mean():.4f}")
        
    elif model_type == 'xgboost':
        print("Using XGBoost models")
        models = models_dict['models']
        test_predictions = np.zeros(len(X_test))
        
        for model in models:
            test_predictions += model.predict_proba(X_test)[:, 1]
        test_predictions /= len(models)
        
        print(f"  Average prediction: {test_predictions.mean():.4f}")
    
    # Prediction statistics
    print(f"\nPrediction Statistics:")
    print(f"  Min:    {test_predictions.min():.4f}")
    print(f"  Max:    {test_predictions.max():.4f}")
    print(f"  Mean:   {test_predictions.mean():.4f}")
    print(f"  Median: {np.median(test_predictions):.4f}")
    print(f"  Std:    {test_predictions.std():.4f}")
    
    # Distribution of predictions
    pred_bins = np.histogram(test_predictions, bins=10)[0]
    print(f"\nPrediction Distribution (10 bins):")
    bin_edges = np.linspace(0, 1, 11)
    for i in range(len(pred_bins)):
        print(f"  {bin_edges[i]:.1f}-{bin_edges[i+1]:.1f}: {pred_bins[i]:,} samples")
    
    return test_predictions

def create_submission_file(test_predictions, test_ids, filename='submission.csv'):
    """
    Create submission file in Kaggle format
    """
    print(f"\nCREATING SUBMISSION FILE: {filename}")
    print("="*50)
    
    # Create submission dataframe
    submission_df = pd.DataFrame({
        'id': test_ids,
        'y': test_predictions
    })
    
    # Save to CSV
    submission_df.to_csv(filename, index=False)
    
    print(f"Submission file saved: {filename}")
    print(f"Shape: {submission_df.shape}")
    print(f"Columns: {list(submission_df.columns)}")
    
    # Show first few rows
    print(f"\nFirst 5 rows:")
    print(submission_df.head())
    
    # Show last few rows
    print(f"\nLast 5 rows:")
    print(submission_df.tail())
    
    # Validation checks
    print(f"\nValidation Checks:")
    print(f"  ✓ ID range: {submission_df['id'].min()} to {submission_df['id'].max()}")
    print(f"  ✓ Prediction range: {submission_df['y'].min():.4f} to {submission_df['y'].max():.4f}")
    print(f"  ✓ No missing values: {submission_df.isnull().sum().sum() == 0}")
    print(f"  ✓ Correct shape: {submission_df.shape[0] == len(test_ids)}")
    
    return submission_df

# Generate test predictions using best model
if best_model == 'Ensemble':
    test_predictions = generate_test_predictions(final_model_results, X_test_final, 'ensemble')
elif best_model == 'LightGBM':
    test_predictions = generate_test_predictions(final_model_results, X_test_final, 'lightgbm')
else:
    test_predictions = generate_test_predictions(final_model_results, X_test_final, 'xgboost')

# Create submission file
submission_df = create_submission_file(test_predictions, test_ids, 'submission.csv')

# Visualize prediction distribution
plt.figure(figsize=(12, 5))

# Prediction histogram
plt.subplot(1, 2, 1)
plt.hist(test_predictions, bins=50, alpha=0.7, color='lightblue', edgecolor='black')
plt.title('Test Set Prediction Distribution', fontweight='bold')
plt.xlabel('Prediction Probability')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# Box plot
plt.subplot(1, 2, 2)
plt.boxplot(test_predictions)
plt.title('Test Set Prediction Box Plot', fontweight='bold')
plt.ylabel('Prediction Probability')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n" + "="*60)
print("SUBMISSION READY!")
print("="*60)
print(f"Final model: {best_model}")
print(f"Cross-validation AUC: {best_auc:.4f}")
print(f"Submission file: submission.csv")
print(f"Test predictions generated for {len(test_predictions):,} samples")
print("\nGood luck with your Kaggle submission! 🚀")